# Single File

In [1]:
using SemanticModels.Dubstep
using Cassette
using LightGraphs
using MetaGraphs

loaded


In [2]:
Cassette.canrecurse( ctx::TypeCtx, f::typeof(ODEProblem))  = false
Cassette.canrecurse( ctx::TypeCtx, f::typeof(solve))  = false
Cassette.canrecurse( ctx::TypeCtx, f::typeof(pairs))  = false
Cassette.canrecurse( ctx::TypeCtx, f::typeof(Base.vect)) = false
Cassette.canrecurse( ctx::TypeCtx, f::typeof(OrdinaryDiffEq.ODECompositeSolution)) = false

UndefVarError: UndefVarError: ODEProblem not defined

In [2]:
m = include("sir.jl")

Main.test

In [3]:
using Main.test

In [52]:
mg = typegraph(Main.test,100)

{1474, 1897} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [7]:
for edge in edges(mg)
    @show get_prop(mg,edge,:label)
end

get_prop(mg, edge, :label) = "missing"
get_prop(mg, edge, :label) = Main.test.main
get_prop(mg, edge, :label) = NamedTuple
get_prop(mg, edge, :label) = Base.vect
get_prop(mg, edge, :label) = tuple
get_prop(mg, edge, :label) = length
get_prop(mg, edge, :label) = add_int
get_prop(mg, edge, :label) = Colon()
get_prop(mg, edge, :label) = Base.IteratorSize
get_prop(mg, edge, :label) = axes
get_prop(mg, edge, :label) = iterate
get_prop(mg, edge, :label) = isa
get_prop(mg, edge, :label) = isa
get_prop(mg, edge, :label) = Core.apply_type
get_prop(mg, edge, :label) = similar
get_prop(mg, edge, :label) = LinearIndices
get_prop(mg, edge, :label) = first
get_prop(mg, edge, :label) = ===
get_prop(mg, edge, :label) = not_int
get_prop(mg, edge, :label) = getfield
get_prop(mg, edge, :label) = getindex
get_prop(mg, edge, :label) = setindex!
get_prop(mg, edge, :label) = iterate
get_prop(mg, edge, :label) = iterate
get_prop(mg, edge, :label) = ===
get_prop(mg, edge, :label) = Base.vect
get_prop(mg, edge,

In [8]:
for vertex in vertices(mg)
    @show get_prop(mg,vertex,:label)
end

get_prop(mg, vertex, :label) = ()
get_prop(mg, vertex, :label) = "missing"
get_prop(mg, vertex, :label) = OrdinaryDiffEq.ODECompositeSolution{Float64,2,Array{Array{Float64,1},1},Nothing,Nothing,Array{Float64,1},Array{Array{Array{Float64,1},1},1},DiffEqBase.ODEProblem{Array{Float64,1},Tuple{Float64,Float64},true,Array{Float64,1},DiffEqBase.ODEFunction{true,getfield(Main.test, Symbol("#sir_ode#1")),LinearAlgebra.UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing},Nothing,DiffEqBase.StandardODEProblem},OrdinaryDiffEq.CompositeAlgorithm{Tuple{OrdinaryDiffEq.Tsit5,OrdinaryDiffEq.Rosenbrock23{0,false,DiffEqBase.LinSolveFactorize{typeof(LinearAlgebra.lu!)},DataType}},OrdinaryDiffEq.AutoSwitch{OrdinaryDiffEq.Tsit5,OrdinaryDiffEq.Rosenbrock23{0,false,DiffEqBase.LinSolveFactorize{typeof(LinearAlgebra.lu!)},DataType},Rational{Int64},Float64}},OrdinaryDiffEq.CompositeInterpolationData{DiffEqBase.ODEFunction{true,getfield(Main.test, Symbol("#sir_ode#1")),LinearAlge

In [3]:
"""
escapeHTML(i::String)
Returns a string with special HTML characters escaped: &, <, >, ", '
"""
function escapehtml(i::AbstractString)
    # Refer to http://stackoverflow.com/a/7382028/3822752 for spec. links
    o = replace(i, "&" =>"&amp;")
    o = replace(o, "\""=>"&quot;")
    o = replace(o, "'" =>"&#39;")
    o = replace(o, "<" =>"&lt;")
    o = replace(o, ">" =>"&gt;")
    return o
end

function savedot(io::IOStream, g::MetaDiGraph)
    write(io, "digraph G {\nrankdir=\"LR\";\n")
    for p in props(g)
        write(io, "$(p[1])=$(escapehtml(string(p[2])));\n")
    end

    for v in vertices(g)
        write(io, "$v")
        if length(props(g, v)) > 0
            write(io, " [ ")
        end
        for p in props(g, v)
            key = p[1]
            write(io, "$key=\"$(escapehtml(string(p[2])))\",")
        end
        if length(props(g, v)) > 0
            write(io, "];")
        end
        write(io, "\n")
    end

    for e in edges(g)
        write(io, "$(src(e)) -> $(dst(e)) [ ")
        for p in props(g,e)
            write(io, "$(p[1])=\"$(escapehtml(string(p[2])))\", ")
        end
        write(io, "]\n")
    end
    write(io, "}\n")
end

function savegraph(fn::AbstractString, g::AbstractMetaGraph, ::DOTFormat)
    open(fn, "w") do fp 
        savedot(fp, g)
    end
end

savegraph (generic function with 1 method)

In [48]:
savegraph("sir.dot",mg,DOTFormat())

2

In [59]:
run(`dot -Tsvg -O sir_edit.dot`)

Process(`dot -Tsvg -O sir_edit.dot`, ProcessExited(0))

# Directory

In [5]:
files = readdir("files")

14-element Array{String,1}:
 "ContinuousTimeSIR.jl"                
 "DiscreteStochErlangEpidModel.jl"     
 "DiscreteTimeSIR.jl"                  
 "ExactRecursiveExpressions.jl"        
 "MacroParasiteModel.jl"               
 "NHosts1Vector.jl"                    
 "NHostsMVectors.jl"                   
 "OneHostSEIR1Vect.jl"                 
 "SEIRmodel.jl"                        
 "SIRModel.jl"                         
 "SIRSDynamicsLargePopulationsJulia.jl"
 "SISModel.jl"                         
 "ScalingModel.jl"                     
 "SemiParamSIRJulia.jl"                

In [1]:
using SemanticModels.Dubstep
using Cassette
using LightGraphs
using MetaGraphs

loaded


In [2]:
function escapehtml(i::AbstractString)
    # Refer to http://stackoverflow.com/a/7382028/3822752 for spec. links
    o = replace(i, "&" =>"&amp;")
    o = replace(o, "\""=>"&quot;")
    o = replace(o, "'" =>"&#39;")
    o = replace(o, "<" =>"&lt;")
    o = replace(o, ">" =>"&gt;")
    return o
end

function savedot(io::IOStream, g::MetaDiGraph)
    write(io, "digraph G {\nrankdir=\"LR\";\n")
    for p in props(g)
        write(io, "$(p[1])=$(escapehtml(string(p[2])));\n")
    end

    for v in vertices(g)
        write(io, "$v")
        if length(props(g, v)) > 0
            write(io, " [ ")
        end
        for p in props(g, v)
            key = p[1]
            write(io, "$key=\"$(escapehtml(string(p[2])))\",")
        end
        if length(props(g, v)) > 0
            write(io, "];")
        end
        write(io, "\n")
    end

    for e in edges(g)
        write(io, "$(src(e)) -> $(dst(e)) [ ")
        for p in props(g,e)
            write(io, "$(p[1])=\"$(escapehtml(string(p[2])))\", ")
        end
        write(io, "]\n")
    end
    write(io, "}\n")
end

function savegraph(fn::AbstractString, g::AbstractMetaGraph, ::DOTFormat)
    open(fn, "w") do fp 
        savedot(fp, g)
    end
end

savegraph (generic function with 1 method)

In [3]:
file = "NHosts1Vector.jl"

"NHosts1Vector.jl"

In [ ]:
include("./files/$file")
using Main.Example
mg = typegraph(Main.Example,100)
savegraph("$(file[1:end-3]).dot",mg,DOTFormat())